In [1]:
import math
import pandas as pd
import os
import numpy as np
import random
import connectorx as cx
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
os.chdir(desktop_path)
print("Current Directory:", os.getcwd())

Current Directory: C:\Users\dozeren\Desktop


In [26]:
def batch_query_with_keys(query_template, key_list, batch_size=1000):
    
    # List to store DataFrames from each batch
    result_dfs = []
    
    # Calculate number of batches
    num_batches = len(key_list) // batch_size + (1 if len(key_list) % batch_size > 0 else 0)
    print(f"Total number of batches to process: {num_batches}")
    print(f"Total keys to process: {len(key_list)}")
    
    for i in range(num_batches):
        try:
            print(f"Processing batch {i+1}/{num_batches}...")
            
            # Extract keys for current batch
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(key_list))
            batch_keys = key_list[start_idx:end_idx]
            
            if not batch_keys:
                print("Empty batch, skipping.")
                continue
            
            print(f"Number of keys in this batch: {len(batch_keys)}")
            print(f"First few keys in batch: {batch_keys[:3]}...")
            
            # Format the keys for SQL query
            formatted_keys = ", ".join([f"'{key}'" for key in batch_keys])
            query = query_template.replace(":client_keys", f"({formatted_keys})")
            
            print(f"Query preview (first 200 chars): {query[:200]}...")
            
            # Execute query using global db_url
            print("Executing query via ConnectorX...")
            batch_df = cx.read_sql(db_url, query, return_type="pandas")
            
            print(f"Query completed. Batch shape: {batch_df.shape}")
            
            if batch_df.empty:
                print("Warning: Query returned empty DataFrame")
            else:
                print(f"Sample of data: \n{batch_df.head(2)}")
            
            # Store results
            result_dfs.append(batch_df)
            print(f"Batch {i+1}/{num_batches} completed and added to results")
            
        except Exception as e:
            print(f"Error in batch {i+1}/{num_batches}: {str(e)}")
            # Uncomment to continue despite errors
            # continue
    
    # Combine results
    if result_dfs:
        print(f"Concatenating {len(result_dfs)} DataFrames...")
        final_df = pd.concat(result_dfs, ignore_index=True)
        print(f"Final DataFrame shape: {final_df.shape}")
        return final_df
    else:
        print("No data was collected. Check for errors above.")
        return pd.DataFrame()  # Return empty DataFrame if no results

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Suppress chained assignment warnings
pd.options.mode.chained_assignment = None

# Connection details for Oracle databases
connections = {
    'readonly': {
        'user': 'FD_STG',
        'pass': 'FD_STG',
        'dsn': "NJ01PRDDWDB01.NJ01:1521/DATAWPRO"
    },
    'dbsto_readonly': {
        'user': 'APPDEV',
        'pass': 'readn0wrt',
        'dsn': "NJ01PRDDGUARD01.NJ01:1521/DBSTO_READONLY"
    }
}

# Oracle connection string format for ConnectorX
db_url = "oracle://{user}:{password}@{dsn}".format(
    user=connections['readonly']['user'],
    password=connections['readonly']['pass'],
    dsn=connections['readonly']['dsn']
)

In [5]:
pp_df_raw = pd.read_csv('pp_new.csv')

In [6]:
distinct_tier_4_df = pp_df_raw.groupby('TIER4').agg({
    'MATERIAL_NUMBER': 'nunique'
}).reset_index()

In [7]:
distinct_tier_4_df[distinct_tier_4_df['MATERIAL_NUMBER'] > 4].sort_values('MATERIAL_NUMBER', ascending=False)

,TIER4,MATERIAL_NUMBER
13,4%,228
262,All Sparkling Water,217
8,2%,194
1,0%,170
603,Dairy,132
...,...,...
435,Candles,5
1298,Roast Beef DM,5
1303,Roasted Unshelled Peanuts,5
1306,Round,5


In [9]:
distinct_tier_3_df = pp_df_raw.groupby('TIER3').agg({
    'MATERIAL_NUMBER': 'nunique',
    'AVG_PRICE': 'mean',
    'WEIGHTED_AVG_PRICE': 'median'
}).reset_index()

In [10]:
def calculate_mad_from_median(group, median_col):
    deviations = (group['AVG_PRICE'] - group[median_col]).abs()
    return deviations.median()

In [12]:
median_tier3_prices = distinct_tier_3_df[['TIER3', 'WEIGHTED_AVG_PRICE']].rename(
    columns={'WEIGHTED_AVG_PRICE': 'MEDIAN_PRICE'}
)

# 2. Merge to get median prices alongside original data
df_with_medians = pp_df_raw.merge(median_tier3_prices, on='TIER3')

# 3. Calculate absolute deviations for each row
df_with_medians['ABS_DEVIATION'] = (df_with_medians['AVG_PRICE'] - df_with_medians['MEDIAN_PRICE']).abs()

# 4. Group by TIER3 and get the median of absolute deviations
tier3_mad = df_with_medians.groupby('TIER3')['ABS_DEVIATION'].median().reset_index()
tier3_mad.columns = ['TIER3', 'MEDIAN_ABSOLUTE_DEVIATION']

# 5. Merge back to distinct_tier_3_df
distinct_tier_3_df = distinct_tier_3_df.merge(tier3_mad, on='TIER3', how='left')

In [13]:
distinct_tier_3_df['ROBUST_SD'] = 1.4826 * distinct_tier_3_df['MEDIAN_ABSOLUTE_DEVIATION']

In [14]:
distinct_tier_3_df = distinct_tier_3_df[distinct_tier_3_df['MATERIAL_NUMBER'] > 4]

In [16]:
distinct_tier_3_df = distinct_tier_3_df.rename(columns={'MATERIAL_NUMBER': 'SIZE_OF_TIER3', 'AVG_PRICE': 'AVG_PRICE_TIER3', 'WEIGHTED_AVG_PRICE': 'MEDIAN_PRICE_TIER3'})

In [20]:
def find_optimal_tau(tier3_category, initial_tau=1.0, target_percentile=0.8):
    cat_info = distinct_tier_3_df[distinct_tier_3_df['TIER3'] == tier3_category].iloc[0]
    median_price = cat_info['MEDIAN_PRICE_TIER3']
    robust_sd = cat_info['ROBUST_SD']

    cat_products = pp_df_raw[pp_df_raw['TIER3'] == tier3_category]
    if len(cat_products) < 5:  # Need at least 10 products to find a decile meaningfully
        return initial_tau
    
    # Sort products by price (assuming higher price correlates with premium/higher revenue)
    cat_products_sorted = cat_products.sort_values(by='AVG_PRICE', ascending=False)
    
    # Calculate the number of products to include in the top revenue decile
    n_top_decile = max(1, int(len(cat_products) * (1 - target_percentile)))

    price_threshold = cat_products_sorted.iloc[n_top_decile-1]['AVG_PRICE']
    
    if robust_sd > 0:  # Avoid division by zero
        optimal_tau = (price_threshold - median_price) / robust_sd
        # Ensure tau is positive - if it's negative, it means the threshold is below the median
        optimal_tau = max(0.5, optimal_tau)  # Set a minimum of 0.5 standard deviations
        return optimal_tau
    else:
        return initial_tau

In [21]:
# Apply the function to each TIER3 category
distinct_tier_3_df['OPTIMAL_TAU'] = distinct_tier_3_df['TIER3'].apply(
    lambda x: find_optimal_tau(x, initial_tau=1.0, target_percentile=0.90)
)
# Calculate the premium price threshold for each category
distinct_tier_3_df['PREMIUM_PRICE_THRESHOLD'] = (
    distinct_tier_3_df['MEDIAN_PRICE_TIER3'] + 
    distinct_tier_3_df['OPTIMAL_TAU'] * distinct_tier_3_df['ROBUST_SD']
)

In [22]:
def filter_premium_products(df_original, df_thresholds):
    df_with_premium_flag = df_original.copy()
    df_with_premium_flag['IS_PREMIUM'] = 0
    threshold_dict = dict(zip(df_thresholds['TIER3'], df_thresholds['PREMIUM_PRICE_THRESHOLD']))
    for tier3, threshold in threshold_dict.items():
        mask = (df_with_premium_flag['TIER3'] == tier3) & (df_with_premium_flag['AVG_PRICE'] > threshold)
        df_with_premium_flag.loc[mask, 'IS_PREMIUM'] = 1
    

    df_premium = df_with_premium_flag[df_with_premium_flag['IS_PREMIUM'] == 1].copy()
    
    return df_premium

In [23]:
df_premium_sku = filter_premium_products(pp_df_raw, distinct_tier_3_df)

print(f"Total products: {len(pp_df_raw)}")
print(f"Premium products: {len(df_premium_sku)} ({len(df_premium_sku)/len(pp_df_raw)*100:.2f}%)")

premium_by_category = df_premium_sku.groupby('TIER3').size().reset_index(name='premium_count')
total_by_category = pp_df_raw.groupby('TIER3').size().reset_index(name='total_count')

premium_analysis = premium_by_category.merge(total_by_category, on='TIER3')
premium_analysis['premium_percentage'] = (premium_analysis['premium_count'] / premium_analysis['total_count'] * 100).round(2)
premium_analysis = premium_analysis.sort_values('premium_percentage', ascending=False)

print("\nTop categories by premium percentage:")
print(premium_analysis.head(10))

Total products: 12737
Premium products: 616 (4.84%)

Top categories by premium percentage:
                               TIER3  premium_count  total_count  \
51            Dough and Wrappers FRZ              1            5   
59                     Farmed Shrimp              1            6   
133                 Racks and Roasts              1            6   
41   Creams Ointments and Powders BB              1            8   
192                  Whole and Split              1           10   
90                  Juice Shots Rfgd              3           31   
77                      Greek Yogurt             22          234   
53        Dried Pasta and Noodles SS             14          151   
158                  Sparkling Water             21          228   
37                       Crackers SS             14          153   

     premium_percentage  
51                20.00  
59                16.67  
133               16.67  
41                12.50  
192               10.00  
90  

In [24]:
material_numbers_premium = df_premium_sku['MATERIAL_NUMBER'].unique()
material_numbers_premium = material_numbers_premium.tolist()

In [25]:
premium_sales_query = '''
SELECT DISTINCT olf.MATERIAL_NUMBER, LATEST_DESCRIPTION, TIER1, TIER2, TIER3, TIER4,
                sum(case when pd.pricing_unit = 'LB' then olf.inv_actual_weight else olf.inv_actual_quantity end) as units,
                count(distinct SALE_ID),
                count(distinct CUSTOMER_KEY),
                sum(INV_ACTUAL_PRICE),
                max(pmld.UNRESTRICTED_USE_INVENTORY),
   max(pmld.UNRESTRICTED_USE_INVENTORY_CS),
   max(pmld.OOS_CURRENT_STATUS),
   max(pmld.DAYS_ON_HAND)
from fd_dw.DW_OLF_L2Y olf
    INNER JOIN fd_dw.product_material_dim pmd
        ON pmd.MATERIAL_NUMBER = olf.MATERIAL_NUMBER
        AND olf.plant = pmd.plant
        AND olf.sales_org = pmd.sales_org
        AND olf.distribution_channel = pmd.distribution_channel
    INNER JOIN fd_dw.PRODUCT_DIM pd
        on olf.product_key = pd.PRODUCT_KEY
        INNER JOIN FD_DW.PRODUCT_MATERIAL_LIVE_DIM pmld
   ON pmld.MATERIAL_NUMBER = olf.MATERIAL_NUMBER
   AND olf.PLANT = pmld.PLANT
   AND olf.sales_org = pmld.sales_org
   AND olf.distribution_channel = pmld.distribution_channel
WHERE ORDER_CREATION_DATE_KEY BETWEEN 20240805 AND 20250805
    AND olf.MG_ORDER_CODE NOT IN ('M', 'MO')
    AND olf.REG_NON_CAN_FLAG = 'Y'
    AND pmd.ACTIVE_PRODUCT_FLAG = 'Y'
and olf.MATERIAL_NUMBER IN :client_keys
group by olf.MATERIAL_NUMBER, LATEST_DESCRIPTION, TIER1, TIER2, TIER3, TIER4'''

In [27]:
df_description_premium = batch_query_with_keys(premium_sales_query, material_numbers_premium)

Total number of batches to process: 1
Total keys to process: 616
Processing batch 1/1...
Number of keys in this batch: 616
First few keys in batch: [300502807, 300504359, 400220058]...
Query preview (first 200 chars): 
SELECT DISTINCT olf.MATERIAL_NUMBER, LATEST_DESCRIPTION, TIER1, TIER2, TIER3, TIER4,
                sum(case when pd.pricing_unit = 'LB' then olf.inv_actual_weight else olf.inv_actual_quantity end) ...
Executing query via ConnectorX...
Query completed. Batch shape: (616, 14)
Sample of data: 
  MATERIAL_NUMBER                LATEST_DESCRIPTION    TIER1       TIER2  \
0       200210146           PEACH 40-50CT CASE [EA]  Produce       Fruit   
1       200302044  CAULIFLOWER CUT RICE 10OZ [EACC]  Produce  Vegetables   

                    TIER3                         TIER4   UNITS  \
0             Stone Fruit        Peaches and Nectarines   365.0   
1  Vegetables Value Added  Cut Cauliflower and Broccoli  8695.0   

   COUNT(DISTINCTSALE_ID)  COUNT(DISTINCTCUSTOMER_KEY)  

In [28]:
df_description_premium['MATERIAL_NUMBER'] = df_description_premium['MATERIAL_NUMBER'].astype('int64')

In [29]:
df_premium_sku = df_premium_sku.merge(df_description_premium, on='MATERIAL_NUMBER', how='left')

In [31]:
df_premium_sku.to_excel('premium_products_2025.xlsx')